In [ ]:
import numpy as np
import scipy
from scipy.optimize import minimize, LinearConstraint, Bounds
from scipy.linalg import lu, det

In [ ]:
p = 1000
paramTrue = [1,.1]

In [ ]:
# function that makes covariance matrix
def Sigma(param, p):
    I = np.eye(p) #dtype=np.float32)
    O = np.ones([p,p]) #dtype=np.float32)
    return I*param[0]+(O-I)*param[1]

In [ ]:
# check if SigmaTrue is positive semidefinite
SigmaTrue = Sigma(paramTrue, p)
np.linalg.eig(SigmaTrue)[0].all() >= 0

True

In [ ]:
# check if cholesky decomposition works
SigmaHalf = np.linalg.cholesky(SigmaTrue)
np.dot(SigmaHalf, SigmaHalf.T)

array([[1. , 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 1. , 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 1. , ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 1. , 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 1. , 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 1. ]])

In [ ]:
# generate random vector Y = SigmaHalf * U
U = np.random.normal(0, 1, p).reshape(p, 1)
Y = np.dot(SigmaHalf, U)
Y.shape # dimension of Y

(1000, 1)

In [ ]:
# define log likelihood function
def logLikelihood(param, Y):
    
    p = len(Y)
    sig = Sigma(param,p)
    inv_Sigma = np.linalg.inv(sig)
    
    log_det_Sigma = 2 * np.sum(np.log(np.diag(np.linalg.cholesky(sig)))) # reduce computation complexity

    # det_Sigma = np.linalg.det(sig)
    # log_det_Sigma = np.log(det_Sigma)
        
    return 0.5 * Y.T @ inv_Sigma @ Y + 0.5 * log_det_Sigma


In [ ]:
# define log likelihood function
def logLikelihood(param, Y):
    
    p = len(Y)
    sig = Sigma(param,p)
    inv_Sigma = np.linalg.inv(sig)
    
    mean = np.array([0]*p)

    y = scipy.stats.multivariate_normal(mean, sig)
    
  return -np.sum(y.logpdf(data))

* Sigma psd <-> The min value of eigenvalue should be same or bigger than 0. 
* Optimizaition methods used when there is inequilaty constraints: SLSQP, COBYLA, trust-constr


In [ ]:
param0 = paramTrue

# Define the constraint function
def constraint1(param, Y):
    p = len(Y)
    sig = Sigma(param, p)
    eigenvalues = np.linalg.eig(sig)[0]
    return eigenvalues.min() - 1e-6 # Constraints: eigenvalues >= 0

                
# Define the optimization problem
problem = {
    'fun': logLikelihood,         # Objective function: minimize the minimum eigenvalue
    'x0': paramTrue,            # Initial guess for the parameters
    'args': (Y,),             # Additional arguments for the objective function 
    'constraints': {'type': 'ineq', 'fun': constraint1, 'args':(Y,)},  # Constraint function: eigenvalues >= 0
    'method': 'SLSQP'        
}

result = minimize(**problem)


estimated_params = result.x

LinAlgError: ignored

In [ ]:
estimated_params